# Oasis Reinsurance Test Tool v0.6

This notebook allows example reinsurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Included
* Basic direct insurance modelling (not the complete engine included in Flamingo)
    * Buildings, Other buildings contents and BI
    * Blanket limit
    * Blanket deductible
    * Site limit
    * Site deductible 
* FAC treaties
* Quota share
* Surplus share
* Per-risk
* Cat XL treaties

## Not included
* Attachement basis
* Aggregate terms
* Multiple portfolios in a single set of input files

## Test cases

A selection of test cases can be found in the examples folder.

* simple_loc_FAC: Single location level fac.
* simple_pol_FAC: Single policy level fac.
* simple_acc_FAC: Single account level fac.
* multiple_FAC: Multiple facs at sameinuring level.
* simple_QS: Single quota share with no risk limits.
* loc_SS: Single surplus share at location level.
* pol_SS: Single surplus share at location level.
* acc_SS: Single surplus share at location level.
* multiple_SS: Multiple surplus shares at same inuring level.
* loc_limit_QS: Single quota share with location level risk limits.
* pol_limit_QS: Single quota share with policy level risk limits.
* acc_limit_QS: Single quota share with account level risk limits.
* multiple_QS_1:  Two quota shares at same inuring level.
* multiple_QS_2:  Two quota shares at different inuring levels.
* simple_CAT_XL: Single cat XL.
* multiple_CAT_XL: Two cat XLs at different inuring levels.


## Input files 

Files must be named the following:
* account.csv
* location.csv
* ri_info.csv
* ri_scope.csv


## Validation Rules
* Risk levels cannot be mixed in a single reinsurance scope
* Values in the scope file must link to rows in ACC/LOC exposure file set.
* QS always has non-specific scope
* SS always has specific scope
* Reinsurance types cannot be combined in an inuring layer

In [ ]:
# Notebook and Python 
%config IPCompleter.greedy=True

# Standard Python libraries
import io
import json
import os

# 3rd party Python libraries
import jupyter_helper
import pandas as pd
import six
    
# Oasis imports
import oasislmf._data as static_data 
from oasislmf.model_preparation import oed
from oasislmf.utils.deterministic_loss import (
    generate_oasis_files,
    generate_losses,
)

In [ ]:
jupyter_helper.file_uploader('examples/uploaded')

In [ ]:
# Load the OED for a worked example, in this case FM Test Case 23.

# Note that only the currently used fields are shown unless show_all is set to True. 
oed_dir = os.path.abspath('examples_old/loc_SS')

# Account file
oed_account_file = os.path.join(oed_dir, "account.csv")
if not os.path.exists(oed_account_file):
    print("Path does not exist: {}".format(oed_account_file))
    exit(1)
account_df = pd.read_csv(oed_account_file)

# Location file
oed_location_file = os.path.join(oed_dir, "location.csv")
if not os.path.exists(oed_location_file):
    print("Path does not exist: {}".format(oed_location_file))
    exit(1)
location_df = pd.read_csv(oed_location_file)
(
    ri_info_df, 
    ri_scope_df, 
    do_reinsurance
) = oed.load_oed_dfs(oed_dir, show_all=False)

In [ ]:
# View/edit the account data. 
account_grid = jupyter_helper.show_df(account_df)
account_grid

In [ ]:
# View/edit the location data.
location_grid = jupyter_helper.show_df(location_df)
location_grid

In [ ]:
# View/edit the reinsurance info data.
ri_info_grid = jupyter_helper.show_df(ri_info_df)
ri_info_grid

In [ ]:
# View/edit the reinsurance scope data.
ri_scope_grid = jupyter_helper.show_df(ri_scope_df)
ri_scope_grid

In [ ]:
# Pick up any edits in the grid before running the analysis
account_df = account_grid.get_changed_df()
location_df = location_grid.get_changed_df()
ri_info_df = ri_info_grid.get_changed_df()
ri_scope_df = ri_scope_grid.get_changed_df()

direct_oasis_files_dir = os.path.abspath('run_reinsurance')

if not os.path.exists(direct_oasis_files_dir):
    os.mkdir(direct_oasis_files_dir)

account_df.to_csv(os.path.join(direct_oasis_files_dir, "account.csv"), index=False)
location_df.to_csv(os.path.join(direct_oasis_files_dir, "location.csv"), index=False)
ri_info_df.to_csv(os.path.join(direct_oasis_files_dir, "ri_info.csv"), index=False)
ri_scope_df.to_csv(os.path.join(direct_oasis_files_dir, "ri_scope.csv"), index=False)

data_fp = static_data.__path__._path[0]    
srcacctocan_trans_fp = os.path.join(data_fp, 'MappingMapToOED_CanAccA.xslt')
srcexptocan_trans_fp = os.path.join(data_fp, 'MappingMapToOED_CanLocA.xslt')
output_dir = os.path.join(direct_oasis_files_dir, "output")
(ri_layers, xref_descriptions) = generate_oasis_files(
    direct_oasis_files_dir, output_dir, 
    srcexptocan_trans_fp, srcacctocan_trans_fp)
loss_factor = 0.5
net_losses = generate_losses(
    output_dir, xref_descriptions, loss_percentage_of_tiv=loss_factor, ri_layers=ri_layers)
print("Ran {} inuring layers".format(len(net_losses) - 1))
print("Losses for:")
for key in net_losses.keys():
    print("\t{}".format(key))

In [ ]:
# View the direct losses.
key = 'Direct'
net_losses[key]

In [ ]:
# View the losses for the first inuring layer.
key = 'Inuring_priority:1 - Risk_level:LOC'
net_losses[key]

## Optional Steps


In [ ]:
# Automated testing on reinsurance examples
!python3.6 -m pytest -sp no:flaky -v tests/test_reinsurance.py

In [ ]:
# Write outputs to CSV for download.
for (description, net_loss) in net_losses.items():
    filename = "output_{}.csv".format(description.replace(' ', '_'))
    net_loss.to_csv(filename, index=False)